In [1]:
!pip install -qq adapters datasets

In [2]:
!pip uninstall -y transformers adapters peft trl accelerate tokenizers
!pip install transformers==4.51.3
!pip install adapters==1.2.0
!pip install accelerate==0.26.0

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: adapters 1.2.0
Uninstalling adapters-1.2.0:
  Successfully uninstalled adapters-1.2.0
Found existing installation: accelerate 0.26.0
Uninstalling accelerate-0.26.0:
  Successfully uninstalled accelerate-0.26.0
Found existing installation: tokenizers 0.21.4
Uninstalling tokenizers-0.21.4:
  Successfully uninstalled tokenizers-0.21.4


  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl (2.5 MB)

   ---------------------------------------- 0/2 [tokenizers]
   ---------------------------------------- 0/2 [tokenizers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------

In [7]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, load_dataset
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, RobertaConfig, TrainingArguments, EvalPrediction
from adapters import AutoAdapterModel, AdapterTrainer

In [4]:
data_tom = load_dataset("rotten_tomatoes")
data_tom.num_rows

{'train': 8530, 'validation': 1066, 'test': 1066}

In [5]:
print(data_tom)
print(data_tom["train"][2])

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})
{'text': 'effective but too-tepid biopic', 'label': 1}


In [8]:
cora = pd.read_csv("../Hyperedges/Cora_dataset.csv")

le = LabelEncoder()
cora['labels'] = le.fit_transform(cora['topic2']) 

cora_ft = Dataset.from_pandas(cora[["Abstract", "labels"]])
cora_ft = cora_ft.rename_column("Abstract", "text")

In [9]:
cora

,paper_id,filenames,Title,Author,Abstract,BoW,topic2,Abstract_W2V,bert_embedding,cluster_embs,labels
0,35,['http_##www.cis.ohio-state.edu#lair#TechRepor...,Evolutionary Module Acquisition (1993) Coevolv...,"Angeline and Pollack Angeline, P. and Pollack,...","Angeline, P., Saunders, G. and Pollack, J. (19...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Genetic_Algorithms,[ 0.00257568 0.01624146 -0.01265259 0.129727...,[ 6.13429993e-02 -3.51591222e-02 -8.50764588e-...,[ 2.61604437e-03 2.69720778e-02 -3.40913646e-...,1
1,40,['http_##www.bioele.nuee.nagoya-u.ac.jp#wec#pa...,Dynamic Control of Genetic Algorithms using Fu...,Michael A. LEE Hideyuki TAKAGI,This paper proposes using fuzzy logic techniqu...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Genetic_Algorithms,[ 2.27050781e-02 4.92503271e-02 5.16006462e-...,[ 6.13822490e-02 -9.98590514e-03 -7.81871602e-...,[ 1.28001701e-02 3.91861163e-02 -1.01053238e-...,1
2,114,['ftp_##ftp.cs.colorado.edu#users#baveja#Paper...,Learning to Act using Real-Time Dynamic Progra...,Andrew G. Barto Steven J. Bradtke Satinder P. ...,"fl The authors thank Rich Yee, Vijay Gullapall...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",Reinforcement_Learning,[-0.01200994 0.03754203 0.00502043 0.093253...,[ 5.97590730e-02 -6.56006113e-03 -7.85881057e-...,[ 2.11583525e-02 -5.75699378e-03 -1.89218428e-...,4
3,117,['http_##www.cs.duke.edu#~mlittman#docs#gmdp.ps'],Generalized Markov Decision Processes: Dynamic...,Csaba Szepesvari Michael L. Littman,The problem of maximizing the expected total d...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Reinforcement_Learning,[ 0.01002187 0.00597592 0.02532374 0.099285...,[ 6.34309798e-02 -2.35317685e-02 -8.46546218e-...,[ 1.14918491e-02 3.67332175e-02 7.56769720e-...,4
4,128,['ftp_##ftp.cs.colorado.edu#users#baveja#Paper...,Reinforcement Learning Algorithms for Average-...,Satinder P. Singh,Reinforcement learning (RL) has become a centr...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Reinforcement_Learning,[ 0.01000858 -0.0008307 0.03161937 0.086836...,[ 7.14411512e-02 -2.84189135e-02 -8.62762332e-...,[ 7.57048698e-03 3.90252322e-02 5.19991573e-...,4
...,...,...,...,...,...,...,...,...,...,...,...
2363,1154276,['ftp_##ftp.icsi.berkeley.edu#pub#techreports#...,Scatter-partitioning RBF network for function ...,NaN,segmentation: Preliminary results Abstract. Sc...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Neural_Networks,[-0.00997262 0.04042348 0.02706561 0.063081...,[ 4.68115099e-02 -3.41062658e-02 -8.05736557e-...,[ 1.29470171e-03 1.26969337e-03 -3.27199996e-...,2
2364,1154520,['ftp_##ftp.cs.rutgers.edu#pub#farach#SpliceJu...,that fits the asymptotics of the problem. Refe...,NaN,1] D. Aldous and P. Shields. A diffusion limit...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Neural_Networks,[-0.01303735 0.01778633 0.03181899 0.080227...,[ 4.66245227e-02 -1.68257095e-02 -7.54518732e-...,[-3.11294547e-03 -2.28649098e-02 -3.65703143e-...,2
2365,1154524,['http_##www.cs.rutgers.edu#~uli#cs671#ruttenb...,Abstract,NaN,This paper is a scientific comparison of two c...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Rule_Learning,[ 1.03734573e-02 8.40596855e-03 4.98973355e-...,[ 5.32725081e-02 -9.84915346e-03 -7.38246143e-...,[ 6.29070401e-03 3.94592695e-02 -5.06803021e-...,5
2366,1154525,['http_##www.cs.rutgers.edu#~uli#cs671#ertl-cc...,Instructions,M. Anton Ertl Andreas Krall,Paper and BibTeX entry are available at http:/...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Rule_Learning,[-4.73944703e-03 1.88433565e-02 1.70901790e-...,[ 7.37561807e-02 -2.18447298e-02 -7.84817040e-...,[ 4.75611426e-02 -3.42832354e-04 -1.49267381e-...,5


In [7]:
cora_ft = cora_ft.train_test_split(test_size=0.2, seed=42)
test_valid = cora_ft["test"].train_test_split(test_size=0.5, seed=42)

dataset_dict = {
    "train": cora_ft["train"],
    "validation": test_valid["train"],
    "test": test_valid["test"]
}

dataset = DatasetDict(dataset_dict)

In [8]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
    return tokenizer(
        batch["text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

dataset = dataset.map(encode_batch, batched=True)
dataset = dataset.with_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 237/237 [00:00<00:00, 637.97 examples/s]


In [9]:
config = RobertaConfig.from_pretrained("roberta-base",num_labels=7)
model = AutoAdapterModel.from_pretrained("roberta-base", config=config)

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
n_labels = len(set(cora["topic2"]))
id2label = {i: lab for i, lab in enumerate(le.classes_)}
# Add a new adapter
model.add_adapter("cora_fn", config="seq_bn")

# Add a matching classification head
model.add_classification_head(
    "cora_fn",
    num_labels=n_labels,
    id2label=id2label,
  )

# Activate the adapter
model.train_adapter("cora_fn")

There are adapters available but none are activated for the forward pass.


In [11]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=9,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    logging_steps=50,
    output_dir="./cora_fn_out",
    overwrite_output_dir=True,
    report_to=[],
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"accuracy": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [12]:
trainer.train()

Step,Training Loss
50,1.882500
100,1.843200
150,1.356100
200,0.978700
250,0.936200
300,0.744800
350,0.761400
400,0.689700
450,0.643700
500,0.607400


TrainOutput(global_step=1071, training_loss=0.7617467208848058, metrics={'train_runtime': 7229.0951, 'train_samples_per_second': 2.358, 'train_steps_per_second': 0.148, 'total_flos': 1149150069080064.0, 'train_loss': 0.7617467208848058, 'epoch': 9.0})

In [13]:
trainer.evaluate()

{'eval_loss': 0.5739740133285522,
 'eval_accuracy': 0.8185654008438819,
 'eval_runtime': 23.8145,
 'eval_samples_per_second': 9.952,
 'eval_steps_per_second': 0.336,
 'epoch': 9.0}

## **Guardar y Leer nuevos embeddings**

In [ ]:
from adapters import AutoAdapterModel
from transformers import RobertaTokenizer
import torch

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Load base model
model = AutoAdapterModel.from_pretrained("roberta-base")
adapter_path = "./cora_fn_out/checkpoint-1000/cora_fn" # coger checkpoint para generar embeddings

model.load_adapter(
    adapter_path,
    load_as="cora_fn",
    set_active=True 
)

model.set_active_adapters("cora_fn")

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['heads.default.3.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
There are adapters available but none are activated for the forward pass.


Active adapters: Stack[cora_fn]


In [ ]:
from torch.utils.data import DataLoader
def encode_abstracts(texts, batch_size=32, max_length=128):
    all_embeddings = []

    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)

    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(
                batch,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            inputs = {k: v for k, v in inputs.items()}

            # Forward pass through the (encoder only), ignoring the head
            outputs = model.base_model(**inputs)  
            embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
            all_embeddings.append(embeddings.cpu())

    return torch.cat(all_embeddings, dim=0)


In [ ]:
cora = pd.read_csv("../Hyperedges/Cora_dataset.csv")

le = LabelEncoder()
cora['labels'] = le.fit_transform(cora['topic2']) 

cora_ft = Dataset.from_pandas(cora[["Abstract", "labels"]])
cora_ft = cora_ft.rename_column("Abstract", "text")

# Coger los abstracts
abstracts = cora_ft["text"]

# Compute embeddings
embeddings = encode_abstracts(abstracts, batch_size=16)
print("Embeddings shape:", embeddings.shape)


Embeddings shape: torch.Size([2368, 768])


In [ ]:
embeddings_np = embeddings.numpy()
cora["cls_embedding"] = list(embeddings_np)

# Guardo el dataset con los nuevos embeddings
cora.to_pickle("cora_with_embeddings.pkl")

In [ ]:
# He hecho lo mismo fine-tuneando los embeddings de RoBERTa para los abstracts de PubMed
# Load embeddings de PubMed

loaded_embeddings = np.load("./PubMed_embeddings.npz")["embeddings"]
print("Embeddings shape:", loaded_embeddings.shape)

Embeddings shape: (19716, 768)


array([[-0.2475476 ,  0.6143033 ,  1.0025382 , ..., -0.05929684,
        -0.02109905,  0.5139046 ],
       [-0.04540623,  0.7384806 ,  0.7458422 , ...,  0.29003197,
        -0.01394354,  0.41901493],
       [-0.30693296,  0.4041639 ,  0.9072337 , ...,  0.17131016,
        -0.04385674,  0.5062645 ],
       ...,
       [-0.04761544,  0.61029446,  0.85678405, ...,  0.01582604,
         0.35217664,  0.4982269 ],
       [-0.4546575 ,  0.3067275 ,  0.08663608, ..., -1.2792853 ,
         0.34755304, -0.08672211],
       [-0.19107838,  0.5639358 ,  0.3021389 , ..., -1.3055748 ,
         0.23849142, -0.0692755 ]], dtype=float32)